In [1]:
import pandas as pd
from pyproj import Proj
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn import model_selection as cross_validation
from sklearn.metrics import mean_squared_error
import seaborn as sns
sns.set(color_codes=True)
import collections

%matplotlib inline

In [ ]:
ls data/

In [ ]:
data_path = "data/merged/bronx_brooklyn_manhattan_queens_statenisland_2003_2016.csv"
df = pd.read_csv(data_path, low_memory = False)
df.head()

In [ ]:
no_dups = df.drop_duplicates()
no_dups.shape

In [ ]:
df.shape

In [ ]:
df[['bbl','zipcode','latitude','longitude']].to_csv("bbls_and_zips.csv", index = False)

In [ ]:
x = df[df['year_built'] != df['yearbuilt']]
x[x['year_built'] == 0.0].shape

In [ ]:
df[['gross_sqft_pluto','sale_price','price_per_sqft']].head()

In [ ]:
df = df[df['price_per_sqft'] != 0.0]
df.shape

In [ ]:
df = df[df['price_per_sqft'] >= 10]
df = df[df['price_per_sqft'] <= 5000]
df.shape

In [ ]:
count = collections.Counter(df['price_per_sqft'].astype(int))
count

In [ ]:
outliers = df.loc[df['price_per_sqft'].astype(int).isin([1,2,3,4,5,6912,7655,7949,10315,12089,14568])]
outliers[['latitude','longitude','price_per_sqft','sale_price','year_built','sale_date']].to_csv("outliers.csv")

In [ ]:
#output distribution of target variable to visualize in Tableau
import csv
x = df['price_per_sqft']
count = collections.Counter(x.astype(int))
with open("price_per_sqft_counts.csv",'w') as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(['Price Per Sqft', 'Frequency'])
    for key, count in count.items():
        writer.writerow([key, count])

In [ ]:
#output distribution of sale price to visualize in Tableau
import csv
x = df['sale_price']
count = collections.Counter(x.astype(int))
with open("sale_price_counts.csv",'w') as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(['Sale Price', 'Frequency'])
    for key, count in count.items():
        writer.writerow([key, count])

In [ ]:
def drop_cols(data, cols):
    return data.drop(cols, axis = 1)

In [ ]:
df = drop_cols(df, ['zonemap','sale_date','sale_price','year_built'])

In [ ]:
from sklearn import model_selection
def split_data(data):
    '''
    Splits data into training and test sets (0.8/0.2)
        Args: 
            data: Pandas dataframe
        Returns:
            data_train: Pandas dataframe used for training
            data_test: Pandas dataframe used for testing
    
    '''
    #Convert 'int64' into float; otherwise, sklearn throws a warning message
    columns = data.columns.values
    non_float = []
    for col in columns:
        if data[col].dtype != np.float64:
            non_float.append(col)
    for col in non_float:
        data[col] = data[col].astype(float)
    #drop NaN for crucial columns
    data= data.dropna(how = 'any', subset = ['price_per_sqft'])   
    #Split the data
    rs = model_selection.ShuffleSplit(train_size = 0.8, test_size=.2, random_state = 1, n_splits = 1)

    for train, test in rs.split(data):
        train_index = train
        test_index = test
    data_train = data.ix[train_index,:]
    data_test = data.ix[test_index,:]
    data_train.reset_index(drop=True, inplace=True)
    data_test.reset_index(drop=True, inplace=True)
    return data_train, data_test


In [ ]:
data_train, data_test = split_data(df)

In [ ]:
def fill_na(data_train, data_test):
    '''
    Fills NaN values with the mean of the column. Note we have already created dummy variables
    for columns with missing values.
    
    Args:
        data_train: Pandas dataframe used for training.
        data_test: Pandas dataframe used for testing.
    Returns:
        data_train: Pandas dataframe with no NaN values, ready for modeling.
        data_test: Pandas dataframe with no NaN values, ready for testing.
    
    '''
    data_train = data_train.apply(lambda x: x.fillna(x.mean()),axis=0)
    data_test = data_test.apply(lambda x: x.fillna(x.mean()),axis=0)
    return data_train, data_test

In [ ]:
data_train, data_test = fill_na(data_train, data_test)

In [ ]:
print(data_train.shape, data_test.shape)


In [ ]:
cols = list(data_train.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('price_per_sqft')) #Remove b from list
data_train = data_train[cols+['price_per_sqft']]
data_test = data_test[cols+['price_per_sqft']]
data_train.columns.values

In [ ]:
data_train = data_train.drop(['public_recycling_bins_dist'], axis = 1)
data_test = data_test.drop(['public_recycling_bins_dist'], axis = 1)

In [ ]:
X_train = data_train.ix[:,:-1]
y_train = data_train.ix[:,-1]
X_test = data_test.ix[:,:-1]
y_test = data_test.ix[:,-1]
regr = linear_model.LinearRegression()
#regr.fit(X_train, y_train)
#mse = mean_squared_error(y_test, regr.predict(X_test))
#print('Mean_squared_error', mse)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

RF_reg_final = RandomForestRegressor(n_estimators=100, n_jobs = -1)
RF_reg_final.fit(X_train, y_train)
print(mean_squared_error(y_test, RF_reg_final.predict(X_test)))

In [ ]:
bbls = []
file = open("data/Queens Light Rail BBL.csv", 'rb')
for line in file:
    bbls.append(line)
bbls = [float(i) for i in bbls]
bbls

In [ ]:
finance = pd.read_csv("finance_queens.csv")
finance.shape

In [ ]:
queens = pd.read_csv("data/nyc_pluto_16v1/QN.csv", low_memory = False)
queens.head()

In [ ]:
buildings_2 = pd.read_csv("pluto_finance_test.csv")
buildings_2.shape

In [ ]:
buildings_3 = pd.read_csv("data/merged/queens_2003_2016.csv")
buildings_3['price_per_sqft'].isnull().sum()

In [ ]:
buildings_3 = buildings_3.drop_duplicates()
buildings_3.shape

In [ ]:
df.bbl.isnull().sum()

In [ ]:
buildings_3.bbl.isnull().sum()

In [ ]:
buildings = pd.merge(queens, finance, how='right',
        left_on='BBL', right_on = 'bbl')
buildings.shape

In [ ]:
buildings_2 = buildings_2.drop_duplicates()
buildings_2.shape

In [ ]:
affected_properties = finance.loc[finance['bbl'].isin(bbls)]
affected_properties.shape

In [ ]:
list(X_test['bbl'])

In [ ]:
feature_importance =  RF_reg_final.feature_importances_
indices = np.argsort(feature_importance)[::-1][:27]

feature_dct = {}
# Print the feature ranking
print("Feature ranking:")

for f in range(20):
    feature_dct[X_test.columns.values[indices][f]] = feature_importance[indices[f]]
feature_dct

In [ ]:
from collections import OrderedDict
from operator import itemgetter

feature_dct = OrderedDict(sorted(feature_dct.items(), key=itemgetter(1), reverse = True))

In [ ]:
feature_dct.keys()

In [ ]:
data_train[['price_per_sqft'] + list(feature_dct.keys())].corr()

In [ ]:
neg_features = feature_dct.copy()
pos_features = feature_dct.copy()

#Use correlation matrix to determine which features are negatively correlated with our target variable
negs = ['gross_sqft_pluto','unitstotal']
for key in neg_features.keys():
    if key in negs:
        neg_features[key] = -neg_features[key]
    else:
        neg_features[key] = 0
for key in pos_features.keys():
    if key in negs:
        pos_features[key] = 0

In [ ]:
data_test['predicted'] = RF_reg_final.predict(X_test)
data_test['percent_difference'] = 100*(np.abs(data_test['predicted'] - data_test['price_per_sqft']).astype(float) / data_test['price_per_sqft'])
data_test[['yearbuilt','price_per_sqft','predicted','percent_difference']]

In [ ]:
acc = 100 * (data_test[data_test['percent_difference'] < 10.0].shape[0]/ data_test.shape[0])
acc

In [ ]:
results_df = pd.read_csv("preliminary_results.csv")
results_df

In [ ]:
import merge_pluto_finance_new as mpf

In [ ]:
queens_pluto = mpf.read_in_pluto(["queens"])

In [ ]:
queens_pluto.bbl.astype(int).isnull().sum()

In [ ]:
queens_finance = mpf.read_in_finance(["queens"], list(range(2003, 2017)))

In [ ]:
queens_finance.bbl.isnull().sum()

In [ ]:
print(queens_finance.shape)
print(queens_pluto.shape)

In [ ]:
print(queens_pluto.bbl.dtype)
print(queens_finance.bbl.dtype)

In [ ]:
queens_dtm = read_in_dtm(["queens"])

In [ ]:
import os
def read_in_dtm(boros, data_dir = 'data/dtm',
        filename = 'DTM_0316_Condo_Units.csv'):
    """
    Reads in the Digital Tax Map dataset and returns a dataframe with mapping
    from borough and condo number to unit BBL for the specified boroughs.

    Args:
        list(string) boros: list of all the boroughs to pull dtm data for
        string data_dir: a relative path as a string to folder containing the
            dtm data in csv format
        string filename: the name of the file containing the dtm condo unit data
    Returns:
        Pandas DataFrame
    """
    columns = ['CONDO_BORO', 'CONDO_NUMB', 'UNIT_BLOCK',
               'UNIT_LOT', 'UNIT_BBL', 'UNIT_DESIG']
    boro_names = ['manhattan', 'bronx', 'brooklyn', 'queens', 'statenisland']
    boro_codes = dict(zip(boro_names, range(1,6)))
    dtm = pd.read_csv(os.path.join(data_dir, filename), usecols=columns)
    dtm.columns = [col.strip().lower() for col in dtm.columns]
    dtm = dtm.dropna(subset = ['unit_bbl', 'condo_boro', 'condo_numb'])
    dtm.unit_bbl = dtm.unit_bbl.astype('int64')
    dtm = dtm.loc[dtm.condo_boro.isin(
        [boro_codes.get(boro) for boro in boros])]
    return dtm

In [ ]:
columns = ['CONDO_BORO', 'CONDO_NUMB', 'UNIT_BLOCK',
               'UNIT_LOT', 'UNIT_BBL', 'UNIT_DESIG']
data_dir = 'data/dtm'
filename = 'DTM_0316_Condo_Units.csv'
queens_dtm = pd.read_csv(os.path.join(data_dir, filename), usecols=columns)

In [ ]:
queens_dtm.isnull().sum()

In [ ]:
filepath = "data/open_nyc/subwaydist.csv"
subwaydist = pd.read_csv(filepath)
filepath = "data/open_nyc/some_dist_metrics.csv"
other_distances = pd.read_csv(filepath)

In [ ]:
len(pd.unique(other_distances.bbl)) == other_distances.shape[0]

In [ ]:
len(pd.unique(subwaydist.bbl)) == subwaydist.shape[0]

In [ ]:
finance_condo_updated = mpf.get_finance_condo_lot(pluto = queens_pluto,
        finance = queens_finance, dtm = queens_dtm)

In [ ]:
finance_condo_updated.bbl_pluto.isnull().sum()

In [ ]:
queens_pluto.bbl.isnull().sum()

In [ ]:
buildings = pd.merge(queens_pluto, finance_condo_updated, how='right',
        left_on='bbl', right_on = 'bbl_pluto',
        suffixes=['_pluto', '_finance'])

In [ ]:
buildings.shape

In [ ]:
buildings["price_per_sqft"] = buildings["sale_price"].astype('float64') / buildings["gross_sqft_pluto"]
buildings = buildings[ buildings["price_per_sqft"].notnull()]
buildings = buildings[ buildings["price_per_sqft"] > 0.]

In [ ]:
buildings.bbl_pluto 

In [ ]:
buildings.bbl

In [ ]:
queens_pluto.bbl

In [ ]:
finance_condos_only.bbl_finance

In [ ]:
queens_finance.columns

In [ ]:
queens_pluto.columns

In [ ]:
queens_dtm.columns

In [ ]:
finance_condo_updated.columns

In [ ]:
print(queens_finance.drop_duplicates().shape == queens_finance.shape)
print(queens_pluto.drop_duplicates().shape == queens_pluto.shape)
print(finance_condos_only.drop_duplicates().shape == finance_condos_only.shape)

In [ ]:
finance_condos_only.shape[0] + len(standard_bbls) 

In [ ]:
bbl_mappings.drop_duplicates().shape

In [ ]:
queens_finance.drop_duplicates(['bbl', 'sale_year']).shape == queens_finance.shape

In [ ]:
queens_finance.shape

In [ ]:
queens_finance.columns

In [ ]:
queens_finance["sale_year"] = [d.year for d in queens_finance.sale_date]

In [ ]:
print(finance_condo_updated.drop_duplicates().shape)
print(finance_condo_updated.shape)

In [ ]:
finance_condo_updated.columns

In [ ]:
bbl_mappings.bbl_finance.drop_duplicates().shape

In [ ]:
dtm_cols_to_keep = ['unit_bbl', 'condo_boro', 'condo_numb']
pluto_cols_to_keep = ['bbl', 'block', 'borocode', 'condono']

finance_condos_only = pd.merge(queens_finance, queens_dtm[dtm_cols_to_keep],
    how='inner', left_on=['bbl'], right_on=['unit_bbl'])

# for condos: finance.bbl == finance_condos_only.unit_bbl
#             finance_condos_only.bbl_pluto == pluto.bbl
finance_condos_only = pd.merge(queens_pluto[pluto_cols_to_keep],
    finance_condos_only, how='inner',
    left_on=['borocode', 'block', 'condono'],
    right_on=['condo_boro', 'block', 'condo_numb'],
    suffixes=['_pluto', '_finance'])

finance_condos_only = finance_condos_only[
        ['bbl_pluto', 'bbl_finance']].drop_duplicates()
         # duplicates only if a bbl is listed in multiple years
finance_condos_only = finance_condos_only.loc[lambda df:
        np.floor(df.bbl_pluto / 1e4) == np.floor(df.bbl_finance / 1e4)]

# get a list of bbls that are not condos (same in pluto and finance)
standard_bbls = list(set(queens_finance.bbl).difference(
                    set(finance_condos_only.bbl_finance)))
# combine condo bbls that differ with standard bbls that are the same
bbl_mappings = finance_condos_only.append(pd.DataFrame.from_dict(
    {'bbl_pluto': standard_bbls, 'bbl_finance': standard_bbls}
))
bbl_mappings = bbl_mappings.reset_index(drop = True)

finance_condo_updated = pd.merge(queens_finance, bbl_mappings,
    how='left', left_on='bbl', right_on='bbl_finance')


In [ ]:
# finance condo updated: remove bbl/block/bbl_finance
# retain only bbl_pluto to match with pluto.bbl in merge
finance_condo_updated = finance_condo_updated.drop(
    ['block', 'bbl', 'bbl_finance'], axis=1)

# Remove duplicate bbls by returning only the most recent sales data
# for each BBL and year
finance_condo_updated = finance_condo_updated.reset_index()
finance_condo_updated["sale_year"] = [d.year for d in
    finance_condo_updated.sale_date]
grouped = finance_condo_updated.groupby(['bbl_pluto', 'sale_year'])
max_idx_by_bbl = grouped['sale_price'].idxmax().values
finance_condo_updated = finance_condo_updated.loc[max_idx_by_bbl]

In [ ]:
queens_finance_raw = pd.DataFrame()
data_dir = "data/finance_sales"
years = list(range(2003, 2017))
boros = ['queens']

for year in years:
        for borough in boros:
            print("Pulling Finance data for {}_{}".format(year, borough))
            boro_year = mpf.read_in_boro_year_data(borough, year, data_dir)
            queens_finance_raw = queens_finance_raw.append(boro_year)

In [ ]:
print(finance_condos_only.shape)
print(finance_condos_only.drop_duplicates().shape)
print(finance_condos_only.drop_duplicates('bbl_finance').shape)

In [ ]:
np.where(bbl_mappings.bbl_finance.duplicated(keep=False))

In [ ]:
print(finance_condo_updated.shape)
print(finance_condo_updated.drop_duplicates().shape)
print(finance_condo_updated.drop_duplicates(['bbl_pluto', 'sale_year']).shape)

In [ ]:
finance_condo_updated.columns

In [ ]:
queens_pluto.columns

In [ ]:
buildings = pd.merge(queens_pluto, finance_condo_updated, how='right',
        left_on='bbl', right_on = 'bbl_pluto',
        suffixes=['_pluto', '_finance'])

In [ ]:
buildings.shape

In [ ]:
buildings.shape

In [ ]:
buildings["price_per_sqft"] = buildings.sale_price.astype('float64') / buildings.gross_sqft_pluto
buildings = buildings.loc[ buildings.price_per_sqft.notnull()]
buildings = buildings.loc[ buildings.price_per_sqft > 0.]

In [ ]:
all(buildings.bbl == buildings.bbl_pluto)

In [ ]:
buildings.columns

In [181]:
import merge_pluto_finance_new as mpf
from importlib import reload
reload(mpf)

<module 'merge_pluto_finance_new' from '/Users/jacqueline/Desktop/reveal-estate/merge_pluto_finance_new.py'>

In [182]:
boros = ['all']
years = ['all']
if years == ["all"]:
    years = list(range(2003, 2017))
if boros == ["all"]:
    boros = ["manhattan", "brooklyn", "queens", "bronx", "statenisland"]

In [183]:
# Convert to lowercase and remove spaces in borough names
boros = ["".join(boro.lower().split()) for boro in boros]

In [184]:
print("Getting PLUTO data for: {}".format(boros))
pluto = mpf.read_in_pluto(boros)

Getting PLUTO data for: ['manhattan', 'brooklyn', 'queens', 'bronx', 'statenisland']


In [185]:
print("Getting Finance data for: {} and {}".format(boros, years))
finance = mpf.read_in_finance(boros, years)
print("Getting DTM Condo Unit data for: {}".format(boros))
dtm = mpf.read_in_dtm(boros)

Getting Finance data for: ['manhattan', 'brooklyn', 'queens', 'bronx', 'statenisland'] and [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
Pulling Finance data for 2003_manhattan
Pulling Finance data for 2003_brooklyn
Pulling Finance data for 2003_queens
Pulling Finance data for 2003_bronx
Pulling Finance data for 2003_statenisland
Pulling Finance data for 2004_manhattan
Pulling Finance data for 2004_brooklyn
Pulling Finance data for 2004_queens
Pulling Finance data for 2004_bronx
Pulling Finance data for 2004_statenisland
Pulling Finance data for 2005_manhattan
Pulling Finance data for 2005_brooklyn
Pulling Finance data for 2005_queens
Pulling Finance data for 2005_bronx
Pulling Finance data for 2005_statenisland
Pulling Finance data for 2006_manhattan
Pulling Finance data for 2006_brooklyn
Pulling Finance data for 2006_queens
Pulling Finance data for 2006_bronx
Pulling Finance data for 2006_statenisland
Pulling Finance data for 2007_manhattan
Pull

In [186]:
pluto_mn = mpf.read_in_pluto(["manhattan"])
pluto_bk = mpf.read_in_pluto(["brooklyn"])
pluto_qn = mpf.read_in_pluto(["queens"])
pluto_bx = mpf.read_in_pluto(["bronx"])
pluto_si = mpf.read_in_pluto(["statenisland"])

In [187]:
dtm_mn = mpf.read_in_dtm(["manhattan"])
dtm_bk = mpf.read_in_dtm(["brooklyn"])
dtm_qn = mpf.read_in_dtm(["queens"])
dtm_bx = mpf.read_in_dtm(["bronx"])
dtm_si = mpf.read_in_dtm(["statenisland"])

In [188]:
print(pluto_mn.shape, pluto_mn.drop_duplicates().shape)
print(pluto_bk.shape, pluto_bk.drop_duplicates().shape)
print(pluto_bx.shape, pluto_bx.drop_duplicates().shape)
print(pluto_qn.shape, pluto_qn.drop_duplicates().shape)
print(pluto_si.shape, pluto_si.drop_duplicates().shape)


(40257, 39) (40257, 39)
(264930, 39) (264930, 39)
(82816, 39) (82816, 39)
(310646, 39) (310646, 39)
(113715, 39) (113715, 39)


In [189]:
finance_mn = mpf.read_in_finance(["manhattan"], years)
finance_bk = mpf.read_in_finance(["brooklyn"], years)
finance_qn = mpf.read_in_finance(["queens"], years)
finance_bx = mpf.read_in_finance(["bronx"], years)
finance_si = mpf.read_in_finance(["statenisland"], years)

Pulling Finance data for 2003_manhattan
Pulling Finance data for 2004_manhattan
Pulling Finance data for 2005_manhattan
Pulling Finance data for 2006_manhattan
Pulling Finance data for 2007_manhattan
Pulling Finance data for 2008_manhattan
Pulling Finance data for 2009_manhattan
Pulling Finance data for 2010_manhattan
Pulling Finance data for 2011_manhattan
Pulling Finance data for 2012_manhattan
Pulling Finance data for 2013_manhattan
Pulling Finance data for 2014_manhattan
Pulling Finance data for 2015_manhattan
Pulling Finance data for 2016_manhattan
Pulling Finance data for 2003_brooklyn
Pulling Finance data for 2004_brooklyn
Pulling Finance data for 2005_brooklyn
Pulling Finance data for 2006_brooklyn
Pulling Finance data for 2007_brooklyn
Pulling Finance data for 2008_brooklyn
Pulling Finance data for 2009_brooklyn
Pulling Finance data for 2010_brooklyn
Pulling Finance data for 2011_brooklyn
Pulling Finance data for 2012_brooklyn
Pulling Finance data for 2013_brooklyn
Pulling Fin

In [190]:
dfs = [pluto, pluto_mn, pluto_bk, pluto_qn, pluto_bx, pluto_si, 
      finance, finance_mn, finance_bk, finance_qn, finance_bx, finance_si]

for d in dfs: 
    compare_block = pd.Series([x[1:6] for x in d.bbl], index=d.index).astype(int)
    check_blocks = d.loc[lambda df: df.block != compare_block]
    print(check_blocks.shape[0])

0
0
0
0
0
0
0
0
0
0
0
0


In [191]:
finance.shape[0] == (finance_mn.shape[0] + finance_bk.shape[0] + 
                     finance_bx.shape[0] + finance_qn.shape[0] + finance_si.shape[0])

True

In [192]:
finance_condo_updated = mpf.get_finance_condo_lot(pluto = pluto,
        finance = finance, dtm = dtm)

In [193]:
finance_condo_updated_mn = mpf.get_finance_condo_lot(pluto = pluto_mn,
        finance = finance_mn, dtm = dtm_mn)
finance_condo_updated_bk = mpf.get_finance_condo_lot(pluto = pluto_bk,
        finance = finance_bk, dtm = dtm_bk)
finance_condo_updated_qn = mpf.get_finance_condo_lot(pluto = pluto_qn,
        finance = finance_qn, dtm = dtm_qn)
finance_condo_updated_bx = mpf.get_finance_condo_lot(pluto = pluto_bx,
        finance = finance_bx, dtm = dtm_bx)
finance_condo_updated_si = mpf.get_finance_condo_lot(pluto = pluto_si,
        finance = finance_si, dtm = dtm_si)

In [198]:
dfs = [finance_condo_updated, finance_condo_updated_mn, finance_condo_updated_bk,
      finance_condo_updated_bx, finance_condo_updated_qn, finance_condo_updated_si]

for d in dfs: 
    compare_block = pd.Series([x[1:6] for x in d.bbl_pluto], index=d.index).astype(int)
    check_blocks = d.loc[lambda df: df.block != compare_block]
    print(check_blocks.shape[0])

0
0
0
0
0
0


In [199]:
finance_condo_updated = finance_condo_updated.drop(
        ['block', 'bbl_finance'], axis=1)
finance_condo_updated_mn = finance_condo_updated_mn.drop(
        ['block', 'bbl_finance'], axis=1)
finance_condo_updated_bk = finance_condo_updated_bk.drop(
        ['block', 'bbl_finance'], axis=1)
finance_condo_updated_qn = finance_condo_updated_qn.drop(
        ['block', 'bbl_finance'], axis=1)
finance_condo_updated_bx = finance_condo_updated_bx.drop(
        ['block', 'bbl_finance'], axis=1)
finance_condo_updated_si = finance_condo_updated_si.drop(
        ['block', 'bbl_finance'], axis=1)

In [200]:
print(finance.shape)
print(finance_condo_updated.shape)

(1341451, 8)
(789932, 8)


In [201]:
print("Finance:{} PLUTO:{} DTM:{}".format(finance.shape,
    pluto.shape, dtm.shape))
finance_condo_updated = mpf.get_finance_condo_lot(pluto = pluto,
    finance = finance, dtm = dtm)
print("Finance updated:{}".format(finance_condo_updated.shape))

Finance:(1341451, 8) PLUTO:(812364, 39) DTM:(239263, 6)
Finance updated:(789932, 10)


In [205]:
print("Merging PLUTO with updated Dept. of Finance data")
buildings = pd.merge(pluto, finance_condo_updated, how='right',
    left_on='bbl', right_on = 'bbl_pluto',
    suffixes=['_pluto', '_finance'])
buildings["price_per_sqft"] = (buildings.sale_price.astype('float64') /
    buildings.gross_sqft_pluto)
buildings = buildings[ buildings.price_per_sqft.notnull()]
buildings = buildings[ buildings.price_per_sqft > 0.]
print(buildings.shape)

Merging PLUTO with updated Dept. of Finance data
(508134, 50)


In [206]:
buildings.columns

Index(['borough', 'block_pluto', 'schooldist', 'council', 'ltdheight',
       'splitzone', 'bldgclass', 'landuse', 'easements', 'ownertype',
       'comarea', 'resarea', 'numbldgs', 'numfloors', 'unitsres', 'unitstotal',
       'lotfront', 'lotdepth', 'bldgfront', 'bldgdepth', 'proxcode',
       'irrlotcode', 'lottype', 'bsmtcode', 'yearbuilt', 'builtcode',
       'histdist', 'landmark', 'borocode', 'bbl', 'condono', 'xcoord',
       'ycoord', 'latitude', 'longitude', 'gross_sqft_pluto', 'garage',
       'extension', 'countalter', 'block_finance', 'sale_price', 'sale_date',
       'tax_class_at_time_of_sale', 'residential_units', 'commercial_units',
       'total_units', 'bbl_finance', 'bbl_pluto', 'sale_year',
       'price_per_sqft'],
      dtype='object')

In [202]:
print("Finance:{} PLUTO:{} DTM:{}".format(finance_mn.shape,
    pluto_mn.shape, dtm_mn.shape))
finance_condo_updated_mn = mpf.get_finance_condo_lot(pluto = pluto_mn,
    finance = finance_mn, dtm = dtm_mn)
print("Finance updated:{}".format(finance_condo_updated_mn.shape))

Finance:(338528, 8) PLUTO:(40257, 39) DTM:(113655, 6)
Finance updated:(77215, 10)


In [203]:
print("Finance:{} PLUTO:{} DTM:{}".format(finance_qn.shape,
    pluto_qn.shape, dtm_qn.shape))
finance_condo_updated_qn = mpf.get_finance_condo_lot(pluto = pluto_qn,
    finance = finance_qn, dtm = dtm_qn)
print("Finance updated:{}".format(finance_condo_updated_qn.shape))

Finance:(408436, 8) PLUTO:(310646, 39) DTM:(40498, 6)
Finance updated:(282142, 10)


In [204]:
print("Finance:{} PLUTO:{} DTM:{}".format(finance_bk.shape,
    pluto_bk.shape, dtm_bk.shape))
finance_condo_updated_bk = mpf.get_finance_condo_lot(pluto = pluto_bk,
    finance = finance_bk, dtm = dtm_bk)
print("Finance updated:{}".format(finance_condo_updated_bk.shape))

Finance:(371517, 8) PLUTO:(264930, 39) DTM:(56905, 6)
Finance updated:(255912, 10)


In [207]:
data_full = pd.read_csv('data/merged/bronx_brooklyn_manhattan_queens_statenisland_2003_2016.csv', 
                        low_memory = False)

In [212]:
'longitude' in data_full.columns

False

In [136]:
finance.loc[(finance.block == 8731)]

,block,bbl,sale_price,sale_date,tax_class_at_time_of_sale,residential_units,commercial_units,total_units
46383,8731,3087310024,700000,2003-10-27,1,1,0,1
46384,8731,3087310036,0,2003-08-27,1,1,0,1
46385,8731,3087310043,1750000,2003-06-05,1,1,0,1
46386,8731,3087310046,700000,2003-09-16,1,1,0,1
46387,8731,3087310051,0,2003-05-01,1,1,1,2
46388,8731,3087310054,1775000,2003-04-15,1,1,0,1
169454,8731,3087310018,1750000,2004-05-13,1,1,0,1
169455,8731,3087310051,915000,2004-04-09,1,1,1,2
169522,8731,3087310026,900000,2004-07-06,1,2,0,2
296174,8731,3087310018,2200000,2005-07-19,1,1,0,1


In [125]:
data_dir = "data/finance_sales"
finance = pd.DataFrame()
for year in years:
    for borough in boros:
        print("Pulling Finance data for {}_{}".format(year, borough))
        boro_year = mpf.read_in_boro_year_data(borough, year, data_dir)
        boro_year.dropna(how = 'all', inplace = True)
        finance = finance.append(boro_year)

Pulling Finance data for 2003_manhattan
Pulling Finance data for 2003_brooklyn
Pulling Finance data for 2003_queens
Pulling Finance data for 2003_bronx
Pulling Finance data for 2003_statenisland
Pulling Finance data for 2004_manhattan
Pulling Finance data for 2004_brooklyn
Pulling Finance data for 2004_queens
Pulling Finance data for 2004_bronx
Pulling Finance data for 2004_statenisland
Pulling Finance data for 2005_manhattan
Pulling Finance data for 2005_brooklyn
Pulling Finance data for 2005_queens
Pulling Finance data for 2005_bronx
Pulling Finance data for 2005_statenisland
Pulling Finance data for 2006_manhattan
Pulling Finance data for 2006_brooklyn
Pulling Finance data for 2006_queens
Pulling Finance data for 2006_bronx
Pulling Finance data for 2006_statenisland
Pulling Finance data for 2007_manhattan
Pulling Finance data for 2007_brooklyn
Pulling Finance data for 2007_queens
Pulling Finance data for 2007_bronx
Pulling Finance data for 2007_statenisland
Pulling Finance data for 

In [127]:
finance.iloc[0]

borough                                                                      1
neighborhood                                         ALPHABET CITY            
building_class_category           02  TWO FAMILY HOMES                        
tax_class_at_present                                                         1
block                                                                      375
lot                                                                         32
ease-ment                                                                     
building_class_at_present                                                   B9
address                              746 EAST 6 STREET                        
apartment_number                                                              
zip_code                                                                 10009
residential_units                                                            2
commercial_units                                    

In [128]:
processed_data = finance.copy()
# processed_data = processed_data.reset_index(drop = True)
bbl_columns = processed_data[["borough", "block", "lot"]].itertuples()
bbl_formatted = pd.Series(["%01d%05d%04d" % (row.borough, row.block,
    row.lot) for row in bbl_columns], dtype='int64')
processed_data["bbl"] = bbl_formatted.astype(str)

In [130]:
processed_data.iloc[0]

borough                                                                      1
neighborhood                                         ALPHABET CITY            
building_class_category           02  TWO FAMILY HOMES                        
tax_class_at_present                                                         1
block                                                                      375
lot                                                                         32
ease-ment                                                                     
building_class_at_present                                                   B9
address                              746 EAST 6 STREET                        
apartment_number                                                              
zip_code                                                                 10009
residential_units                                                            2
commercial_units                                    

In [119]:
pluto.loc[lambda df: df.bbl == '3087310024']

,borough,block,schooldist,council,ltdheight,splitzone,bldgclass,landuse,easements,ownertype,...,bbl,condono,xcoord,ycoord,latitude,longitude,gross_sqft_pluto,garage,extension,countalter
272772,BK,8731,22,48,0,0,A,1,0.0,NaN,...,3087310024,0,998581.0,151195.0,40.58166,-73.948408,1760.0,0,0,0.0


In [ ]:
finance.loc[finance.bbl == '3087310024']

In [117]:
finance_condo_updated_bk.loc[lambda df: df.bbl_pluto == '3087310024']

,sale_price,sale_date,tax_class_at_time_of_sale,year_built,residential_units,commercial_units,total_units,bbl_pluto,sale_year
24173,700000,2003-10-27,1,1925,1,0,1,3087310024,2003
57879,630000,2004-08-25,2,1931,6,0,6,3087310024,2004
92927,345000,2005-01-14,2,1963,0,0,0,3087310024,2005
127170,335000,2006-10-25,2,1933,0,0,0,3087310024,2006
159316,409000,2007-07-11,2,1920,0,0,0,3087310024,2007
295612,1225000,2013-04-08,2,1910,8,0,8,3087310024,2013
320454,2200000,2014-11-06,4,1950,0,1,1,3087310024,2014
371177,42000000,2015-09-22,4,1903,0,54,54,3087310024,2015


In [89]:
brooklyn = pd.read_csv('data/merged/brooklyn_2003_2016.csv', low_memory = False)

In [90]:
brooklyn.loc[brooklyn.bbl == 3000260033, 'sale_date']

0    2004-06-12
1    2006-04-20
2    2007-09-26
3    2008-08-01
4    2009-12-08
5    2010-10-12
6    2011-12-09
7    2012-12-19
Name: sale_date, dtype: object

In [91]:
bk = set((a,b) for a,b in zip(brooklyn.bbl, brooklyn.sale_date))

In [92]:
len(bk) == brooklyn.shape[0]

True

In [93]:
whole = set((a,b) for a,b in zip(data_full.bbl, data_full.sale_date))

In [94]:
len(whole) == data_full.shape[0]

True

In [95]:
bk.difference(whole)

{(3087310024, '2013-04-08'),
 (3017700096, '2014-12-31'),
 (3055640064, '2012-06-07'),
 (3010510045, '2015-05-27'),
 (3080810058, '2016-05-18'),
 (3013680011, '2005-06-15'),
 (3047680070, '2014-09-03'),
 (3052370010, '2006-08-18'),
 (3018910046, '2011-03-04'),
 (3062650065, '2010-02-16'),
 (3049540061, '2012-03-02'),
 (3084650047, '2004-06-14'),
 (3057680061, '2013-05-02'),
 (3061070055, '2009-09-24'),
 (3014240049, '2007-07-30'),
 (3079660012, '2008-10-22'),
 (3050260034, '2003-11-12'),
 (3018190049, '2007-03-14'),
 (3016560067, '2003-06-19'),
 (3033800012, '2003-04-10'),
 (3033920054, '2004-06-16'),
 (3082580030, '2008-10-08'),
 (3051510018, '2014-03-18'),
 (3004400033, '2007-12-03'),
 (3071700065, '2015-04-01'),
 (3078820049, '2007-07-25'),
 (3046150020, '2015-09-30'),
 (3009000028, '2005-08-25'),
 (3039880013, '2015-11-24'),
 (3041290020, '2008-08-28'),
 (3061820042, '2009-09-10'),
 (3083000040, '2004-09-29'),
 (3031870013, '2003-07-29'),
 (3063730060, '2004-06-17'),
 (3001780028, 

In [139]:
finance.loc[lambda df: df.block == 8731]

,block,bbl,sale_price,sale_date,tax_class_at_time_of_sale,residential_units,commercial_units,total_units
46383,8731,3087310024,700000,2003-10-27,1,1,0,1
46384,8731,3087310036,0,2003-08-27,1,1,0,1
46385,8731,3087310043,1750000,2003-06-05,1,1,0,1
46386,8731,3087310046,700000,2003-09-16,1,1,0,1
46387,8731,3087310051,0,2003-05-01,1,1,1,2
46388,8731,3087310054,1775000,2003-04-15,1,1,0,1
169454,8731,3087310018,1750000,2004-05-13,1,1,0,1
169455,8731,3087310051,915000,2004-04-09,1,1,1,2
169522,8731,3087310026,900000,2004-07-06,1,2,0,2
296174,8731,3087310018,2200000,2005-07-19,1,1,0,1


In [140]:
finance_bk.loc[lambda df: df.block == 8731]

,sale_price,sale_date,tax_class_at_time_of_sale,year_built,residential_units,commercial_units,total_units,block,bbl
24173,700000,2003-10-27,1,1925,1,0,1,8731,3087310024
24174,0,2003-08-27,1,1955,1,0,1,8731,3087310036
24175,1750000,2003-06-05,1,1925,1,0,1,8731,3087310043
24176,700000,2003-09-16,1,1940,1,0,1,8731,3087310046
24177,0,2003-05-01,1,1935,1,1,2,8731,3087310051
24178,1775000,2003-04-15,1,2003,1,0,1,8731,3087310054
25132,1750000,2004-05-13,1,1930,1,0,1,8731,3075930038
25133,915000,2004-04-09,1,1935,1,1,2,8731,3075930048
25200,900000,2004-07-06,1,1935,2,0,2,8731,3067610075
24767,2200000,2005-07-19,1,1930,1,0,1,8731,3088190011


In [96]:
brooklyn.loc[(brooklyn.bbl == 3087310024) & (brooklyn.sale_date == '2013-04-08')]

,ltdheight,splitzone,easements,comarea,resarea,numbldgs,numfloors,unitsres,unitstotal,lotfront,...,proxcode_2,lottype_0,lottype_1,lottype_2,lottype_3,lottype_4,lottype_5,tax_class_at_time_of_sale_1,tax_class_at_time_of_sale_2,tax_class_at_time_of_sale_3
169050,0.0,0.0,0.0,0.0,1760.0,NaN,2.0,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,1,0


In [ ]:
data_dir = "data/nyc_pluto_16v1"
initials = {"manhattan" : "MN", "brooklyn" : "BK", "bronx" : "BX",
        "queens" : "QN", "statenisland" : "SI"}
filename = "{data_dir}/{boro}.csv".format(data_dir = data_dir,
            boro = initials.get('brooklyn'))
pluto_raw_brooklyn = pd.read_csv(filename, low_memory = False)